Copyright (c) Microsoft Corporation.
Licensed under the MIT License.

# Abstractive Summarization using BertSumAbs on CNN/DailyMails Dataset

## Summary

This notebook demonstrates how to fine tune BERT for abstractive text summarization. Utility functions and classes in the NLP Best Practices repo are used to facilitate data preprocessing, model training, model scoring, result postprocessing, and model evaluation.

### Abstractive Summarization
Abstractive summarization is the task of taking an input text and summarizing its content in a shorter output text. In contrast to extractive summarization, abstractive summarization doesn't take sentences directly from the input text, instead, rephrases the input text.

### BertSumAbs

BertSumAbs refers to an BERT-based abstractive summarization algorithm  in [Text Summarization with Pretrained Encoders](https://arxiv.org/abs/1908.08345) with [published examples](https://github.com/nlpyang/PreSumm). It uses the pretrained BERT model as encoder and finetune both encoder and decoder on a specific labeled summarization dataset like [CNN/DM dataset](https://github.com/harvardnlp/sent-summary). 

The figure below shows the comparison of architecture of the original BERT model (left) and BERTSUM (right), which BertSumAbs is built upon. For BERTSUM, a input document is split into sentences, and [CLS] and [SEP] tokens are inserted before and after each sentence. This resulting sequence is followed by the summation of three kinds of embeddings for each token before feeding into the transformer layers. The positional embedding used in BertSumAbs enables input length of more than 512, which is the  maximum input length for BERT model. 

It should be noted that the architecture only shows the encoder part. For decoder, BertSumAbs also uses a transformer with multiple layers and random initialization. As pretrained weights are used in the encoder, there is a mismatch in encoder and decoder which may result in unstable finetuning. Therefore, in fine tuning, BertSumAbs uses seperate optimizers for encoder and decoder, each uses its own scheduling. In text generation, techniques like trigram blocking and beam search can be used to improve model accuracy.
<img src="https://nlpbp.blob.core.windows.net/images/BertForSummarization.PNG">


## Before you start
Set QUICK_RUN = True to run the notebook on a small subset of data and a smaller number of steps. If QUICK_RUN = False, the notebook takes about 5 hours to run on a VM with 4 16GB NVIDIA V100 GPUs.

To run rouge evaluation, please refer to the section of compute_rouge_perl in [summarization_evaluation.ipynb](./summarization_evaluation.ipynb).

In [1]:
%load_ext autoreload
%autoreload 2

In [5]:
QUICK_RUN = False

In [7]:
import os
import shutil
import sys
from tempfile import TemporaryDirectory
import torch

nlp_path = os.path.abspath("../../")
if nlp_path not in sys.path:
    sys.path.insert(0, nlp_path)

from utils_nlp.models.transformers.abstractive_summarization_bertsum import BertSumAbs, BertSumAbsProcessor

from utils_nlp.dataset.cnndm import CNNDMSummarizationDataset
from utils_nlp.eval.evaluate_summarization import get_rouge

import pandas as pd
import scrapbook as sb

## Data Preprocessing

The dataset we used for this notebook is CNN/DM dataset which contains the documents and accompanying questions from the news articles of CNN and Daily mail. The highlights in each article are used as summary. The dataset consits of ~289K training examples, ~11K valiation examples and ~11K test examples. The length of the news articles is 781 tokens on average and the summaries are of 3.75 sentences and 56 tokens on average.

The significant part of data preprocessing only involve splitting the input document into sentences.

In [8]:
# the data path used to save the downloaded data file
DATA_PATH =  TemporaryDirectory().name
# The number of lines at the head of data file used for preprocessing. -1 means all the lines.
TOP_N = 1000
if not QUICK_RUN:
    TOP_N = -1

In [9]:
train_dataset, test_dataset = CNNDMSummarizationDataset(
            top_n=TOP_N, local_cache_path=DATA_PATH, prepare_extractive=False
        )

100%|██████████| 489k/489k [00:07<00:00, 64.4kKB/s] 


In [10]:
torch.save(train_dataset, "/dadendev/nlp-recipes/examples/text_summarization/abstemp/train_dataset_full.pt")

In [11]:
torch.save(test_dataset, "/dadendev/nlp-recipes/examples/text_summarization/abstemp/test_dataset_full.pt")

In [ ]:
len(train_dataset.get_source()[0])

In [ ]:
len(test_dataset.get_target()[0])

## Model Finetuning

In [ ]:
# notebook parameters
# the cache path
CACHE_PATH = TemporaryDirectory().name

# model parameters
MODEL_NAME = "bert-base-uncased"
MAX_POS = 768
MAX_SOURCE_SEQ_LENGTH = 640
MAX_TARGET_SEQ_LENGTH = 140

# mixed precision setting. To enable mixed precision training, follow instructions in SETUP.md. 
FP16 = False
if FP16:
    FP16_OPT_LEVEL="O2"
    
# fine-tuning parameters
# batch size, unit is the number of tokens
BATCH_SIZE_PER_GPU = 3


# GPU used for training
NUM_GPUS = 2

# Learning rate
LEARNING_RATE_BERT=2e-3
LEARNING_RATE_DEC=0.2


# How often the statistics reports show up in training, unit is step.
REPORT_EVERY=10
SAVE_EVERY=500

# total number of steps for training
MAX_STEPS=5e4
WARMUP_STEPS_BERT=20000
WARMUP_STEPS_DEC=10000
    
if QUICK_RUN:
    MAX_STEPS=1e3

   


In [ ]:
# processor which contains the colloate function to load the preprocessed data
processor = BertSumAbsProcessor(cache_dir=CACHE_PATH, max_src_len=MAX_SOURCE_SEQ_LENGTH, max_target_len=MAX_TARGET_SEQ_LENGTH)
# summarizer
summarizer = BertSumAbs(
    processor, cache_dir=CACHE_PATH, max_pos_length=MAX_POS
)

In [ ]:
BATCH_SIZE_PER_GPU*NUM_GPUS

In [ ]:

summarizer.fit(
        train_dataset,
        num_gpus=NUM_GPUS,
        batch_size=BATCH_SIZE_PER_GPU*NUM_GPUS,
        max_steps=MAX_STEPS,
        learning_rate_bert=LEARNING_RATE_BERT,
        learning_rate_dec=LEARNING_RATE_DEC,
        warmup_steps_bert=WARMUP_STEPS_BERT,
        warmup_steps_dec=WARMUP_STEPS_DEC,
        save_every=SAVE_EVERY,
        report_every=REPORT_EVERY,
        fp16=FP16,
)


In [ ]:
summarizer.save_model(MAX_STEPS, os.path.join(CACHE_PATH, "./bertsumabs.pt"))

## Model Evaluation

In [ ]:
# del summarizer
# checkpoint = torch.load("./abstemp/bert-base-uncased_step_1000.pt")
# summarizer.model.load_checkpoint(checkpoint['model'])

In [ ]:
# clear cache
import gc; gc.collect()
torch.cuda.empty_cache()

In [ ]:
TOP_N=128
shortened_dataset= test_dataset.shorten(top_n=TOP_N)
src = shortened_dataset.get_source()
reference_summaries = [" ".join(t).rstrip("\n") for t in shortened_dataset.get_target()]
generated_summaries = summarizer.predict(
    shortened_dataset, batch_size=32, num_gpus=NUM_GPUS
)
assert len(generated_summaries) == len(reference_summaries)


In [ ]:
src[0]

In [ ]:
generated_summaries[0]

In [ ]:
reference_summaries[0]

In [ ]:
RESULT_DIR = TemporaryDirectory().name
rouge_score = get_rouge(generated_summaries, reference_summaries, RESULT_DIR)
print(rouge_score)

In [ ]:
# for testing
sb.glue("rouge_2_f_score", rouge_score['rouge_2_f_score'])

## Clean up temporary folders

In [ ]:
if os.path.exists(DATA_PATH):
    shutil.rmtree(DATA_PATH, ignore_errors=True)
if os.path.exists(CACHE_PATH):
    shutil.rmtree(CACHE_PATH, ignore_errors=True)
if os.path.exists(RESULT_DIR):
    shutil.rmtree(RESULT_DIR, ignore_errors=True)